In [1]:
LANGUAGE = 'de'

In [2]:
import os

import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [3]:
os.getcwd()

'/cluster/scratch/yakram/sbert-copy/1_my_finetuning_scripts/mnr/data_creation'

In [4]:
CURRENT_FILE_LOCATION = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(CURRENT_FILE_LOCATION, '../../turtle_files')

In [5]:
if LANGUAGE == 'en':
    DATA_FILES_PATH = os.path.join(DATA_DIR, 'english_data')
elif LANGUAGE == 'de':
    DATA_FILES_PATH = os.path.join(DATA_DIR, 'german_data')

NEGATIVE_FILE_PATH = os.path.join(DATA_DIR, f'Negative_mined_de.csv')

In [6]:
os.path.isfile(NEGATIVE_FILE_PATH)

True

In [7]:
MemConcept_prefLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_MemConcept_prefLabel.rq'), sep='\t', index_col=[0])
MemConcept_altLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_MemConcept_altLabel.rq'), sep='\t', index_col=[0])
MemConcept_hiddenLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_MemConcept_hiddenLabel.rq'), sep='\t', index_col=[0])
MemConcept_description = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_MemConcept_description.rq'), sep='\t', index_col=[0])
MemConcept_schemes = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_MemConcept_Schemes.rq'), index_col=[0], delimiter='\t')


In [8]:
SkillsHier_prefLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_SkillsHier_prefLabel.rq'), sep='\t', index_col=[0])
SkillsHier_altLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_SkillsHier_altLabel.rq'), sep='\t', index_col=[0])
SkillsHier_hiddenLabel = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_SkillsHier_hiddenLabel.rq'), sep='\t', index_col=[0])
SkillsHier_description = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_SkillsHier_description.rq'), sep='\t', index_col=[0])
SkillsHier_schemes = pd.read_csv(os.path.join(DATA_FILES_PATH, 'query_SkillsHier_Schemes.rq'), index_col=[0], delimiter='\t')


In [9]:
SkillsHier_schemes.columns = MemConcept_schemes.columns

In [10]:
MemConcept_schemes_grouped = MemConcept_schemes.groupby('?a')['?schemes'].apply(list)
SkillsHier_schemes_grouped = SkillsHier_schemes.groupby('?a')['?schemes'].apply(list)

MemConcept_schemes = MemConcept_prefLabel.merge(MemConcept_schemes_grouped, on='?a', how='left')
SkillsHier_schemes = SkillsHier_prefLabel.merge(SkillsHier_schemes_grouped, on='?a', how='left')

In [11]:
def get_concept_scheme(x):
    concept_names = []
    for elem in sorted(x):
        concept_names.append(elem.split('/')[-1])
    return concept_names

def flag_concept_scheme(x):
    if ('skills' in x) or ('skill' in x):
        return 'skill'
    elif ('occupation' in x) or ('occupations' in x):
        return 'occupation'


def get_skilltype_name(x):
    last_uri_word= x.split('/')

    if last_uri_word[-1] == 'knowledge':
        return 'knowledge'
    else:
        return 'skill'

In [12]:
MemConcept_schemes['concept_schemes'] = MemConcept_schemes['?schemes'].progress_apply(get_concept_scheme)
SkillsHier_schemes['concept_schemes'] = SkillsHier_schemes['?schemes'].progress_apply(get_concept_scheme)

Memconcept_SkillsHier_combined = pd.concat([MemConcept_schemes, SkillsHier_schemes], ignore_index=True)

MemConcept_schemes['concept_flag'] = MemConcept_schemes['concept_schemes'].progress_apply(flag_concept_scheme)
Memconcept_SkillsHier_combined['concept_flag'] = Memconcept_SkillsHier_combined['concept_schemes'].progress_apply(flag_concept_scheme)

100%|██████████| 17537/17537 [00:00<00:00, 834880.87it/s]


In [13]:
skills_uri = Memconcept_SkillsHier_combined[Memconcept_SkillsHier_combined['concept_flag'] == 'skill']['?a']

In [14]:
skills_uri.to_csv(os.path.join(DATA_FILES_PATH, 'query_concept_schemes_combined.rq'), sep='\t')

In [15]:
MemConcept_SkillsHier_prefLabel = pd.concat([MemConcept_prefLabel, SkillsHier_prefLabel])
MemConcept_SkillsHier_altLabel = pd.concat([MemConcept_altLabel, SkillsHier_altLabel])
MemConcept_SkillsHier_hiddenLabel = pd.concat([MemConcept_hiddenLabel, SkillsHier_hiddenLabel])
MemConcept_SkillsHier_description = pd.concat([MemConcept_description, SkillsHier_description])

In [16]:
def create_double_pair(df1, df2):

    skills_copy = skills_uri.copy(deep=True)
    skills_copy = pd.DataFrame(skills_copy)

    combined_df = skills_copy.merge(df1, on=['?a'], how='left').merge(df2, on=['?a'], how='left')
    combined_df.drop(columns=['?a'], inplace=True)
    return combined_df

In [17]:
pref_alt_data = create_double_pair(MemConcept_SkillsHier_prefLabel, MemConcept_SkillsHier_altLabel)
pref_hidden_data = create_double_pair(MemConcept_SkillsHier_prefLabel, MemConcept_SkillsHier_hiddenLabel)
pref_description_data = create_double_pair(MemConcept_SkillsHier_prefLabel, MemConcept_SkillsHier_description)
alt_hidden_data = create_double_pair(MemConcept_SkillsHier_altLabel, MemConcept_SkillsHier_hiddenLabel)
alt_description_data = create_double_pair(MemConcept_SkillsHier_altLabel, MemConcept_SkillsHier_description)
hidden_description_data = create_double_pair(MemConcept_SkillsHier_hiddenLabel, MemConcept_SkillsHier_description)

In [18]:
all_data = [pref_alt_data ,pref_hidden_data ,pref_description_data ,alt_hidden_data ,alt_description_data ,hidden_description_data]

In [19]:
for df in all_data:
    df.columns = ['anchor', 'positive_pair']

In [20]:
skills_labels_valid_pairs = pd.concat([pref_alt_data ,pref_hidden_data ,pref_description_data ,alt_hidden_data ,alt_description_data ,hidden_description_data])

In [21]:
skills_labels_valid_pairs.shape

(96974, 2)

In [22]:
skills_labels_valid_pairs.drop_duplicates(inplace=True)
skills_labels_valid_pairs.dropna(inplace=True)

In [23]:
skills_labels_valid_pairs.shape

(27367, 2)

In [24]:
skills_labels_valid_pairs.to_csv(os.path.join(DATA_FILES_PATH, f'../mnr_data/{LANGUAGE}', 'skills_labels_positive_pairs.csv'), sep='\t')

# Adding negatives, in separate file